# 2. Your first API request

In this section we're going to learn how to send a request for information to the Trove API.

API requests are just like normal urls. However, instead of sending us back a web page, they deliver data in a form that computers can understand. We can then use that data in our own programs.

We're going to use the Python [Requests](http://docs.python-requests.org/en/master/) library to handle our API queries, so let's import it now.

In [11]:
import requests

## Getting an API key
Any requests you make to the Trove API need to be authenticated with a 'key'. For non-commercial projects, you just  fill out a simple form and your API key is generated instantly. Follow the instructions in the Trove Help to [obtain your own Trove API Key](http://help.nla.gov.au/trove/building-with-trove/api).

Once you've created a key, you can access it at any time on the 'For developers' tab of your Trove user profile.

Copy your API key now, and paste it in the cell below, between the quotes.

In [12]:
api_key = ''
print('Your API key is: {}'.format(api_key))

Your API key is: 


In [13]:
api_key = '6pi5hht0d2umqcro'
print('Your API key is: {}'.format(api_key))

Your API key is: 6pi5hht0d2umqcro


All search queries to the Trove API start with the same base url. We'll save it as a variable here.

In [3]:
api_search_url = 'https://api.trove.nla.gov.au/result'

Trove API queries are constructed by adding parameters to the base url. Most of the parameters are optional, but a few are mandatory:

* `q` &ndash; 'q' for query, this is where search terms go
* `zone` &ndash; which Trove zone (or zones) do you want to search, use 'all' for everything
* `key` &ndash; your Trove API key

If you don't want to specify a search term, you can just use a space or a plus sign &ndash; '&nbsp;' or '+' &ndash; as the value for `q`. Of course, this means that you're asking for *everything*, so Trove might take a bit longer to respond.

The default output of the API is XML. For most applications it's easier to work with JSON. You set this using the `encoding` parameter.

We'll meet some other parameters later, but for now let's create a Python dictionary to store our basic parameters. The `requests` library will take this dictionary, turn it into a string, and add it to the base url.


In [4]:
params = {
    'q': 'cyclone', # Search for this keyword -- feel free to change!
    'zone': 'newspaper', # Search in the newspaper zone
    'key': api_key,
    'encoding': 'json'
}

Ok, we're now now ready to make our first query!

In [5]:
response = requests.get(api_search_url, params=params)
print(response.url) # This shows us the url that's sent to the API

https://api.trove.nla.gov.au/result?q=+&zone=all&key=6pi5hht0d2umqcro&encoding=json


The `response` variable contains the results in json format. Let's have a look at what they look like.

In [6]:
data = response.json()
import pprint # we can use pprint to nicely format the json
pprint.pprint(data, width=140)

{'response': {'query': '',
              'zone': [{'name': 'list',
                        'records': {'list': [{'creator': 'public:bozzo2',
                                              'description': 'in the 6th Column A.J Kempt and Co',
                                              'id': '29522',
                                              'listItemCount': 4,
                                              'relevance': {'score': '1.0', 'value': 'very relevant'},
                                              'title': "Rockhampton Cordia lmanufacturer's",
                                              'troveUrl': 'http://trove.nla.gov.au/list?id=29522',
                                              'url': '/list/29522'},
                                             {'creator': 'public:yetta',
                                              'id': '38068',
                                              'listItemCount': 2,
                                              'relevance': {'score': 

                                              'relevance': {'score': '1.0', 'value': 'likely to be relevant'},
                                              'title': 'Kakawin',
                                              'troveUrl': 'http://trove.nla.gov.au/work/151240205',
                                              'type': ['Unpublished'],
                                              'url': '/work/151240205',
                                              'versionCount': 1},
                                             {'holdingsCount': 0,
                                              'id': '151240204',
                                              'issued': 1900,
                                              'relevance': {'score': '1.0', 'value': 'likely to be relevant'},
                                              'title': 'Kaweruhan kawisesan',
                                              'troveUrl': 'http://trove.nla.gov.au/work/151240204',
                                

The individual item records are quite deeply nested in the results.

`data['response']['zone']` is a list of dictionaries, one dictionary for each zone.

Each dictionary includes the following values:

* `name` -- the name of the zone
* `records` -- another dictionary containing the search results
    
The `records` dictionary includes the following values:

* `total` -- the number of matching search results in this zone
* `n` -- the number of results in this set
* `s` -- the starting point of this result set
* `next` -- a path which when added to the base API url will return the next set of results
* and of course the results set itself

Somewhat annoyingly the key for the results set is different in different zones, so you need to know what it is in advance.
 
| Zone       | Key       |
| ---------- | --------- |
| list       | `list`    |
| people     | `people`  |
| newspaper  | `article` |
| book       | `work`    |
| article    | `work`    |
| picture    | `work`    |
| map        | `work`    |
| collection | `work`    |
| music      | `work`    |

For example, to get the results set from the newspapers you'd use `records['article']`.

In [7]:
def zone_totals(data):
    '''
    Display totals in each zone.
    '''
    zones = []
    for zone in data['response']['zone']:
        print('{:<15} {:,}'.format(zone['name'], int(zone['records']['total'])))
        zones.append(zone['name'])

zone_totals(data)

list            75,045
people          1,118,336
music           4,722,156
map             616,537
book            20,969,889
collection      443,429
picture         6,350,141
article         152,747,576
newspaper       219,803,744


While the API returns results for the `people` zone, it's not really returning much useful data. Hopefully this will be fixed in the future, but for now you need to use a separate `people` API.

In [8]:
record_labels = {
    'list': 'list',
    'people': 'people',
    'newspaper': 'article',
    'book': 'work',
    'article': 'work',
    'picture': 'work',
    'map': 'work',
    'collection': 'work',
    'music': 'work'
}

In [9]:
def list_titles(zone_name):
    '''
    List the titles in a specified zone.
    eg: list_titles('newspaper')
    '''
    for zone in data['response']['zone']:
        if zone['name'] == zone_name:
            for record in zone['records'][record_labels[zone_name]]:
                if 'heading' in record:
                    print(record['heading'])
                elif 'title' in record:
                    print(record['title'])

In [10]:
list_titles('book')

Rede door den aftredenden gouverneur generaal van Nederlandsch Indie Mr. P. Meijer, uitgesproken in de vergadering van den raad van Nederlandsch Indie, op Maandag 1 Januarij 1872, bij de overgave van het bestuur aan den benoemden gouverneur generaal van Nederlandsch Indie Mr. James Loudon
Xiao bai tu qi yu
Report of Select Committee of the House of Assembly on the City of Adelaide Municipal Loan Bill; together with minutes of proceedings and evidence
Commission of His Excellency Sir G. Grey, K.C.B., together with the royal instructions accompanying the same
Freedom of information / prepared by the Society's Public Issues Committee
[Pieces de théatre]
A Bill for regulating the survey and sales of land belonging to the Crown in the Australian Colonies and New Zealand / Prepared and brought in by Lord Stanley and Mr. George William Hope
Mačka i glušec / Džejms Patterson
Catalogue of the Austral Gallery : a collection of paintings representing the Bushranging era of Australia ; painted